<a href="https://colab.research.google.com/github/LucasFreitas98/Data-Science/blob/main/prediction_bank_customers_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 53.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=993291f301d711158005f152ad49b9f6c2db11be167b4db1bab54a4b7bfbdbee
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
import pandas as pd

In [8]:
bank_customers_churn = pd.read_csv("churn_modeling.csv", sep=",")

In [9]:
bank_customers_churn.head(4)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0


In [10]:
type(bank_customers_churn)

pandas.core.frame.DataFrame

# Transformando o DataFrame do Pandas para Pyspark

In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder.appName("Pandas").getOrCreate()

In [13]:
churn_df = spark.createDataFrame(bank_customers_churn)

In [14]:
type(churn_df)

pyspark.sql.dataframe.DataFrame

In [15]:
churn_df.show(2)

+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|     0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1|83807.86|            1|        0|             1|      112542.58|     0|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
only showing top 2 rows



# Tranformas DataFrame de PySpark para Pandas

In [16]:
pandas = churn_df.toPandas()

In [17]:
type(pandas)

pandas.core.frame.DataFrame

In [18]:
pandas.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [19]:
bank_customers_churn.iloc[1:4,0:4]

,RowNumber,CustomerId,Surname,CreditScore
1,2,15647311,Hill,608
2,3,15619304,Onio,502
3,4,15701354,Boni,699


In [21]:
bank_customers_churn['CreditScore'].describe()

count    10000.000000
mean       650.528800
std         96.653299
min        350.000000
25%        584.000000
50%        652.000000
75%        718.000000
max        850.000000
Name: CreditScore, dtype: float64

In [23]:
agrupado = bank_customers_churn.groupby(['Geography']).size()

In [24]:
print(agrupado)

Geography
France     5014
Germany    2509
Spain      2477
dtype: int64


# Koalas ajuda na conversão de código do Pandas para PySpark

In [28]:
pip install koalas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 8.1 MB/s 


In [29]:
import databricks.koalas as ks

# Usando o método Naive Bayes para prever o churn de clientes em um DataFrame de um banco específico.

In [36]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import *

In [37]:
spark=SparkSession.builder.master("local[*]").appName("kaggle.com").getOrCreate()

In [41]:
churn = spark.read.option("header",True).csv("churn_modeling.csv", sep=",")

In [42]:
formula = RFormula(formula="Exited ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")

In [43]:
churn_trans = formula.fit(churn).transform(churn).select("features","label")

In [45]:
churn_trans.show(4)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(39854,[0,12735,2...|  1.0|
|(39854,[1112,1325...|  0.0|
|(39854,[2223,1210...|  1.0|
|(39854,[3334,1543...|  0.0|
+--------------------+-----+
only showing top 4 rows



In [46]:
churnTreino, churnTeste  = churn_trans.randomSplit([.7,.3])

In [47]:
nb = NaiveBayes(labelCol="label",featuresCol="features")

In [48]:
modelo = nb.fit(churnTreino)

In [49]:
previsao = modelo.transform(churnTeste)

In [50]:
previsao.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(39854,[4,12496,2...|  0.0|[-86.642786113592...|[0.99126308078375...|       0.0|
|(39854,[14,19711,...|  0.0|[-82.221022867518...|[0.99923436696217...|       0.0|
|(39854,[16,19363,...|  1.0|[-83.793819528324...|[0.99916649740193...|       0.0|
|(39854,[17,15302,...|  0.0|[-91.766645640915...|[0.97730664540403...|       0.0|
|(39854,[19,11486,...|  0.0|[-82.915699355757...|[0.99992437525964...|       0.0|
|(39854,[22,18182,...|  0.0|[-91.899341326895...|[0.99744189376875...|       0.0|
|(39854,[24,11425,...|  1.0|[-88.209311972271...|[0.74013358097854...|       0.0|
|(39854,[28,16059,...|  0.0|[-89.733468146907...|[0.99985486437381...|       0.0|
|(39854,[31,16556,...|  0.0|[-88.321359337510...|[0.99064353317804...|       0.0|
|(39854,[35,1482

In [52]:
avaliar = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="label",metricName="accuracy")

In [53]:
resultado = avaliar.evaluate(previsao)

In [54]:
print(resultado)

0.8046480296396092
